# 装饰器

### deco 返回inner函数对象

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [2]:
@deco
def target():
    print('running target()')

In [3]:
target()
target

running inner()


<function __main__.deco.<locals>.inner()>

### 装饰器在定义时就已运行

In [4]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

running register(<function f1 at 0x7febb10017b8>)
running register(<function f2 at 0x7febb07961e0>)


In [5]:
main()

running main()
registry -> [<function f1 at 0x7febb10017b8>, <function f2 at 0x7febb07961e0>]
running f1()
running f2()
running f3()


In [6]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

In [7]:
from dis import dis
dis(f1)

 10           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('running f1()')
              4 CALL_FUNCTION            1
              6 POP_TOP
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


### 闭包/closure

In [8]:
class Averager():
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
    
def make_average():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [9]:
avg = Averager()
print(avg(10), avg(11), avg(12))

10.0 10.5 11.0


In [10]:
avg = make_average()
print(avg(10), avg(11), avg(12))

10.0 10.5 11.0


in averager function, serise is a free variable, 指未在本地作用域中绑定的变量。averager的闭包延伸到函数的作用域之外，包含自由变量series的绑定

In [11]:
print(avg.__code__.co_varnames, avg.__code__.co_freevars)

('new_value', 'total') ('series',)


In [12]:
avg.__closure__

(<cell at 0x7febb1035c18: list object at 0x7febb4968788>,)

In [13]:
avg.__closure__[0].cell_contents

[10, 11, 12]

闭包是一种函数，它会保留定义含函数时存在的自由变量的绑定。值得注意的是，只有嵌套在其他函数中的函数才可能需要处理不在全局作用域中的外部变量。

### 使用nonlocal

下面是更高效率的版本：只存储目前的sum和count，但存在问题。

In [14]:
def make_averager2():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / conut
    return averager

In [15]:
avg = make_averager2()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

这是因为count和total是不可变类型，`count += 1`的作用其实与`count = count + 1`一样，我们在averager的定义体中为count赋值了，这会把count变为局部变量。而在`make_average`中我们只是调用了`series.append`。

对于数字、字符串等不可变类型来说，只能读取不能更新，否则会隐式创建局部变量，就不能保存在闭包中了。

对此问题，Python3中引入了`nonlocal`声明，用于把变量标记为自由变量。

In [16]:
def make_averager3():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [17]:
avg = make_averager3()
avg(10)

10.0

### Decorator

输出函数的运行时间

In [18]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter() # perf_counter: Return the value (in fractional seconds) of a performance counter,
                                 # substitute for time.clock()
        result = func(*args) # func is a free variable
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [19]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [20]:
print('*'*40, "Calling snooze(.123)")
snooze(.123)
print('*'*40, "Calling factorial(6)")
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12383210s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000140s] factorial(1) -> 1
[0.00002200s] factorial(2) -> 2
[0.00003540s] factorial(3) -> 6
[0.00004730s] factorial(4) -> 24
[0.00071190s] factorial(5) -> 120
[0.00075270s] factorial(6) -> 720
6! = 720


由于`@clock`在定义函数时就已经运行了，所以factorial保存的其实是clocked函数的引用：

In [21]:
factorial.__name__

'clocked'

这就是装饰器的典型行为：把被装饰的函数替换成新函数，且二者接受相同的参数，且（通常）返回被装饰的函数本该返回的值，同时进行一些额外的操作。

上述装饰器有一些缺陷：不支持关键字参数，并且遮盖了原有函数的__name__和__doc__属性。下例使用`functools.wraps`装饰器把相关属性从func中复制到clocked中，且能正常处理关键字参数。

In [22]:
import time
import functools

def clock2(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [23]:
@clock2
def snooze(seconds):
    time.sleep(seconds)

@clock2
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

print('*'*40, "Calling snooze(.123)")
snooze(.123)
print('*'*40, "Calling factorial(6)")
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12330842s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000167s] factorial(1) -> 1
[0.00002837s] factorial(2) -> 2
[0.00004363s] factorial(3) -> 6
[0.00005627s] factorial(4) -> 24
[0.00008655s] factorial(5) -> 120
[0.00011396s] factorial(6) -> 720
6! = 720


In [24]:
factorial.__name__

'factorial'

### 标准库中的装饰器

#### `functools.lru_cache`
实现memoization功能，保存“Least Recently Used”的结果。

In [25]:
@clock2
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [26]:
print(fibonacci(6))

[0.00000119s] fibonacci(0) -> 0
[0.00000143s] fibonacci(1) -> 1
[0.00011015s] fibonacci(2) -> 1
[0.00000119s] fibonacci(1) -> 1
[0.00000119s] fibonacci(0) -> 0
[0.00000191s] fibonacci(1) -> 1
[0.00017452s] fibonacci(2) -> 1
[0.00024939s] fibonacci(3) -> 2
[0.00054240s] fibonacci(4) -> 3
[0.00000167s] fibonacci(1) -> 1
[0.00000119s] fibonacci(0) -> 0
[0.00000167s] fibonacci(1) -> 1
[0.00082231s] fibonacci(2) -> 1
[0.00088263s] fibonacci(3) -> 2
[0.00000119s] fibonacci(0) -> 0
[0.00000119s] fibonacci(1) -> 1
[0.00004005s] fibonacci(2) -> 1
[0.00001788s] fibonacci(1) -> 1
[0.00000143s] fibonacci(0) -> 0
[0.00000191s] fibonacci(1) -> 1
[0.00014758s] fibonacci(2) -> 1
[0.00025296s] fibonacci(3) -> 2
[0.00036740s] fibonacci(4) -> 3
[0.00127649s] fibonacci(5) -> 5
[0.00204897s] fibonacci(6) -> 8
8


In [27]:
import functools

# Call lru_cache as a normal function with '()' operator, as lru_cache can accept configuration parameters.
# Parameters of lru_cache:
#     Param maxsize : default: 128, how many results to cache.
#     Param typed : default: False, whether to save different types of parameters seperately.
@functools.lru_cache()
@clock2
def fibonacci2(n):
    if n < 2:
        return n
    return fibonacci2(n-2) + fibonacci2(n-1)

lru_cache使用字典来存储结果，而且键根据调用时传入的定位参数和关键字参数创建，所以被其装饰的函数的所有参数都必须hashable

In [28]:
print(fibonacci2(6))

[0.00000143s] fibonacci2(0) -> 0
[0.00000286s] fibonacci2(1) -> 1
[0.00029802s] fibonacci2(2) -> 1
[0.00000405s] fibonacci2(3) -> 2
[0.00037432s] fibonacci2(4) -> 3
[0.00000405s] fibonacci2(5) -> 5
[0.00045180s] fibonacci2(6) -> 8
8


#### `functools.singledispatch`

In [46]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

此函数的行为：

In [47]:
print(htmlize({412,23}))
print(htmlize(abs))
print(htmlize('Heimlich & Co. \n- a game'))
print(htmlize(dict))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{412, 23}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<pre>&#x27;Heimlich &amp; Co. \n- a game&#x27;</pre>
<pre>&lt;class &#x27;dict&#x27;&gt;</pre>
<pre>42</pre>
<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


使用`@singledispatch`装饰普通函数，使其变成泛函数(generic function)：根据第一个参数的类型，以不同的方式执行相同操作的一组函数(这才是单分派，若是由多个参数选择专门的函数，那就是多分派了)

实现具体如下，把多个函数绑定在一起组成一个泛函数：

In [60]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence) # 只要可能，注册的专门函数应该处理abc，而不是具体实现（如int）
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [61]:
print(htmlize({412,23}))
print(htmlize(abs))
print(htmlize('Heimlich & Co.\n- a game'))
print(htmlize(dict))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{412, 23}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<p>Heimlich &amp; Co.<br>
- a game</p>
<pre>&lt;class &#x27;dict&#x27;&gt;</pre>
<pre>42 (0x2a)</pre>
<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


In [62]:
htmlize('Heimlich & Co.\n- a game')

'<p>Heimlich &amp; Co.<br>\n- a game</p>'

### 装饰化装饰器
让装饰器接受其他参数：创建一个装饰器工厂函数，把参数传给它，返回一个装饰器，然后再把它应用到要装饰的函数上。

In [65]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x7febb0158c80>)
running main()
registry -> [<function f1 at 0x7febb0158c80>]
running f1()


给register提供一个可选的active参数，为False时不注册被装饰的函数

In [69]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' 
              % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
            
        return func
    return decorate

从概念上看，此register不是装饰器，而是装饰器工厂函数。调用它会返回真正的装饰器。

In [71]:
@register(active=False)
def f1():
    print('running f1()')
    
@register()
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x7febb0133620>)
running register(active=True)->decorate(<function f2 at 0x7febb0133378>)


In [73]:
registry # f2 already in registry

{<function __main__.f2()>}

In [75]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x7febb0133730>)


<function __main__.f3()>

In [76]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [77]:
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x7febb0133378>)


<function __main__.f2()>

In [78]:
registry

{<function __main__.f3()>}

参数化clock装饰器

In [88]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

In [89]:
@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.12320089s] snooze(0.123) -> None
[0.12315416s] snooze(0.123) -> None
[0.12389278s] snooze(0.123) -> None


In [91]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze: 0.1235649585723877s
snooze: 0.12310409545898438s
snooze: 0.1237955093383789s


In [92]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.124s
snooze(0.123) dt=0.124s
snooze(0.123) dt=0.124s


 对装饰器的描述可以精简如下：动态地给一个对象添加一些额外的职责。
 事实上，不建议在Python程序中使用函数装饰器来实现“装饰器”模式，最好使用类来表示装饰器和要包装的组件。

### 延伸阅读

代码来自：https://github.com/fluentpython

闭包：http://effbot.org/zone/closure.htm

实现行为良好的装饰器：http://effbot.org/zone/closure.htm

PythonDecoratorLibrary：https://wiki.python.org/moin/PythonDecoratorLibrary